<a href="https://colab.research.google.com/github/Krithin742/image-to-pdf-converter/blob/main/ats_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00


In [1]:
!pip install PyPDF2 docx2txt nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [2]:
import PyPDF2
import docx2txt
import re
import nltk
import string
from nltk.corpus import stopwords
from google.colab import files

In [3]:
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def extract_text_from_pdf(file_path):
    """Extract text from PDF file."""
    text = ""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text


In [5]:
def extract_text_from_docx(file_path):
    """Extract text from DOCX file."""
    return docx2txt.process(file_path)

In [6]:
def clean_text(text):
    """Lowercase, remove punctuation and stopwords."""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = [w for w in text.split() if w not in STOPWORDS]
    return words

In [7]:
def keyword_match_score(resume_words, job_desc_words):
    """Calculate keyword match percentage."""
    resume_set = set(resume_words)
    job_set = set(job_desc_words)
    common = resume_set.intersection(job_set)
    return round(len(common) / len(job_set) * 100, 2) if job_set else 0

In [8]:

def format_score(text):
    """Score formatting quality."""
    score = 100
    if re.search(r'\t', text):
        score -= 5
    if re.search(r'http[s]?:', text):
        score -= 3
    if re.search(r'[^a-zA-Z0-9\s.,]', text):
        score -= 2
    return max(score, 0)

In [9]:
def section_score(text):
    """Check presence of important sections."""
    sections = ["education", "experience", "skills", "projects", "summary"]
    present = sum(1 for sec in sections if sec in text.lower())
    return round((present / len(sections)) * 100, 2)

In [10]:
def ats_score(keyword_score, format_sc, section_sc):
    """Weighted average ATS score."""
    return round((0.4 * keyword_score) + (0.3 * format_sc) + (0.3 * section_sc), 2)


In [18]:
print("📄 Upload Resume (PDF or DOCX)")
uploaded_resume = files.upload()

resume_filename = list(uploaded_resume.keys())[0]

if resume_filename.endswith(".pdf"):
    resume_text = extract_text_from_pdf(resume_filename)
else:
    resume_text = extract_text_from_docx(resume_filename)

print("📄 Upload Job Description (TXT, PDF, or DOCX)")
uploaded_jd = files.upload()

if not uploaded_jd:
    raise ValueError("❌ No job description file uploaded. Please upload a file.")

jd_filename = list(uploaded_jd.keys())[0]

if jd_filename.endswith(".pdf"):
    job_desc_text = extract_text_from_pdf(jd_filename)
elif jd_filename.endswith(".docx"):
    job_desc_text = extract_text_from_docx(jd_filename)
else:  # Assume plain text
    with open(jd_filename, "r", encoding="utf-8") as f:
        job_desc_text = f.read()


📄 Upload Resume (PDF or DOCX)


Saving resume_rithin.pdf to resume_rithin (2).pdf
📄 Upload Job Description (TXT, PDF, or DOCX)


Saving Resume  (1).pdf to Resume  (1) (1).pdf


In [20]:
print("\n=== ATS Analysis Report ===")
print(f"Keyword Match Score: {kw_score}%")
print(f"Formatting Score: {fmt_score}%")
print(f"Section Completeness Score: {sec_score}%")
print(f"FINAL ATS Friendliness Score: {final_score}%")

print("\n📌 Recommendations:")
if kw_score < 70:
    print("- Add more relevant keywords from the job description.")
if fmt_score < 90:
    print("- Avoid using complex formatting (tables, images, special symbols).")
if sec_score < 100:
    print("- Include missing sections like Education, Experience, Skills, or Summary.")


=== ATS Analysis Report ===
Keyword Match Score: 60.48%
Formatting Score: 98%
Section Completeness Score: 100.0%
FINAL ATS Friendliness Score: 83.59%

📌 Recommendations:
- Add more relevant keywords from the job description.
